In [10]:
!python -m pip install xgboost

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
  Using cached https://files.pythonhosted.org/packages/2e/bd/ad3a963b630fa3ee72d1a672fd207263fa0a18113688273afe8298293535/xgboost-0.82.tar.gz
  Stored in directory: /Users/xueyan/Library/Caches/pip/wheels/5d/ea/e9/4aef52d7294badf6bc26728a73d3e001c38e07f7dc3722c5eb
Successfully built xgboost


In [7]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

import xgboost
from xgboost import XGBClassifier 
# from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

#use 3σ to find outliers
def three_sigma(data_series):
    rule = (data_series.mean() - 3 * data_series.std() > data_series) | (data_series.mean() + 3 * data_series.std() < data_series)
    index = np.arange(data_series.shape[0])[rule]
    outliers = data_series.iloc[index]
    return outliers.tolist()

time = train.playtime_forever #type: series
outliers = three_sigma(time)

#delete the outliers
train_out = train.sort_values(by = 'playtime_forever', ascending = False)
train = train_out.drop(train_out.head(7).index)

#convert is_free to 0,1
for u in train.columns:
    if train[u].dtype==bool:
        train[u]=train[u].astype('int')
        
#drop all rows that have any NaN values
train = train.dropna(axis=0,how='any') 
train.head(5)

,id,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews
90,90,32.766667,0,9900.0,RPG,"Single-player,Steam Achievements,Full controll...","Open World,Post-apocalyptic,Exploration,Single...","Dec 7, 2017","9 Nov, 2015",68660.0,27235.0
221,221,31.983333,0,5000.0,"Indie,RPG","Single-player,Steam Achievements,Steam Trading...","RPG,Turn-Based,Turn-Based Combat,Indie,Choices...","Mar 3, 2017","14 Oct, 2015",1651.0,382.0
236,236,28.983333,1,0.0,"Action,Adventure,Indie,RPG,Simulation","Single-player,Steam Trading Cards,Steam Worksh...","Sandbox,Open World,Survival,Adventure,Crafting...","Oct 28, 2017","8 Nov, 2016",3159.0,1645.0
167,167,24.933333,1,0.0,"Adventure,Casual,Free to Play,Indie,RPG,Simula...","Single-player,Steam Achievements,Steam Trading...","Clicker,Free to Play,Singleplayer,Casual,2D,In...","Sep 17, 2015","13 May, 2015",785.0,84.0
347,347,23.633333,0,20800.0,"Action,Adventure","Single-player,Multi-player,Co-op,Steam Trading...","Open World,Shooter,Action,Co-op,Multiplayer,Ta...","Feb 16, 2018","6 Mar, 2017",23338.0,7681.0


In [13]:
train_allnum = train.drop(columns=['genres','categories','tags','purchase_date','release_date'])
train_allnum.head(10)

,id,playtime_forever,is_free,price,total_positive_reviews,total_negative_reviews
90,90,32.766667,0,9900.0,68660.0,27235.0
221,221,31.983333,0,5000.0,1651.0,382.0
236,236,28.983333,1,0.0,3159.0,1645.0
167,167,24.933333,1,0.0,785.0,84.0
347,347,23.633333,0,20800.0,23338.0,7681.0
308,308,22.350000,0,11900.0,330688.0,127502.0
4,4,22.333333,0,4800.0,8841.0,523.0
309,309,20.616667,0,24800.0,24088.0,4546.0
13,13,20.566667,0,3300.0,9659.0,1058.0
54,54,14.550000,0,16000.0,39351.0,3113.0


In [15]:
for u in test.columns:
    if test[u].dtype==bool:
        test[u]=test[u].astype('int')
test_allnum = test.drop(columns=['genres','categories','tags','purchase_date','release_date'])
X_test = test_allnum[['is_free','price','total_positive_reviews','total_negative_reviews']]

In [16]:
# split data into X and y
X = train_allnum[['is_free','price','total_positive_reviews','total_negative_reviews']]
Y = train_allnum['playtime_forever']

# split data into train and test sets
# seed = 7
# test_size = 0.33
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model
model = XGBClassifier()
# model.fit(X_train, y_train)
model.fit(X, Y)

# make predictions for test data
y_pred = model.predict(X_test)
print(y_pred)
# predictions = [round(value) for value in y_pred]
# print(predictions)
# for idx in range(90):
#     y_pred = model.predict()

# evaluate predictions
# accuracy = accuracy_score(y_test, predictions)



[ 0.          0.          0.          0.          0.13333333  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.         11.51666667  0.          0.          0.          0.
  0.          4.21666667  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.5         0.          0.          0.          0.
  1.75        0.          0.5         0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          4.21666667  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
